In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
years = list(range(1995,2024))

In [4]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    data = requests.get(url)

    with open("mvp/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(data.text)

In [6]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year), "r", encoding='utf=8') as f:
            page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year
    
    dfs.append(mvp)

In [7]:
mvps= pd.concat(dfs)
mvps.tail()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
8,9,Stephen Curry,34,GSW,0.0,5.0,1000,0.005,56,34.7,...,6.1,6.3,0.9,0.4,0.493,0.427,0.915,7.8,0.192,2023
9,10,Jimmy Butler,33,MIA,0.0,3.0,1000,0.003,64,33.4,...,5.9,5.3,1.8,0.3,0.539,0.350,0.850,12.3,0.277,2023
10,11,De'Aaron Fox,25,SAC,0.0,2.0,1000,0.002,73,33.4,...,4.2,6.1,1.1,0.3,0.512,0.324,0.780,7.4,0.146,2023
11,12T,Jalen Brunson,26,NYK,0.0,1.0,1000,0.001,68,35.0,...,3.5,6.2,0.9,0.2,0.491,0.416,0.829,8.7,0.175,2023
12,12T,Ja Morant,23,MEM,0.0,1.0,1000,0.001,61,31.9,...,5.9,8.1,1.1,0.3,0.466,0.307,0.748,6.0,0.148,2023


In [8]:
mvps.to_csv("mvps.csv")

In [10]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

driver=webdriver.Chrome()

for year in years:
    url = player_stats_url.format(year) 

    driver.get(url) 
    driver.execute_script("window.scrollTo(1, 10000)")
    time.sleep(2)

    html = driver.page_source
    with open("player/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(html)

In [11]:
dfs = []
for year in years:
        with open("player/{}.html".format(year),"r", encoding="utf-8") as f:
                page = f.read()

        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        player_table = soup.find(id="per_game_stats")
        player = pd.read_html(str(player_table))[0]
        player["Year"] = year
        dfs.append(player)

In [12]:
players = pd.concat(dfs)
players.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
699,535,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023
700,536,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,...,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023
701,537,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,...,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2023
702,538,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,...,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2023
703,539,Ivica Zubac,C,25,LAC,76,76,28.6,4.3,6.8,...,3.1,6.8,9.9,1.0,0.4,1.3,1.5,2.9,10.8,2023


In [13]:
players.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')

In [14]:
players.to_csv("players.csv")

In [16]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

for year in years:
    url = team_stats_url.format(year)

    data = requests.get(url)

    with open("team/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(data.text)

In [17]:
dfs = []

for year in years:
    with open("team/{}.html".format(year), "r", encoding="utf-8") as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id="divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    dfs.append(team)

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id="divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs.append(team)

In [18]:
teams = pd.concat(dfs)
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,57,25,.695,—,110.9,103.8,6.44,1995,Orlando Magic*
1,55,27,.671,2.0,98.2,95.1,2.78,1995,New York Knicks*
2,35,47,.427,22.0,102.8,104.7,-1.92,1995,Boston Celtics*
3,32,50,.390,25.0,101.1,102.8,-1.85,1995,Miami Heat
4,30,52,.366,27.0,98.1,101.2,-3.28,1995,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,51,31,.622,—,116.9,113.0,3.60,2023,Memphis Grizzlies*
14,42,40,.512,9.0,114.4,112.5,1.63,2023,New Orleans Pelicans*
15,38,44,.463,13.0,114.2,114.1,-0.14,2023,Dallas Mavericks
16,22,60,.268,29.0,110.7,118.6,-7.62,2023,Houston Rockets


In [19]:
teams.to_csv("teams.csv")